## Spotify Song-Album Popularity

This is a CS109A project to build a model to predict whether a song will be the most popular one on its album. This notebook allows for the collection of the data.

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd
import time

In [2]:
credentials = json.load(open('credentials.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

In [3]:
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Gather The Data From The API

In [4]:
columns = ['track_id', 
           'album_id', 
           'track_number', 
           'track_count', 
           'duration', 
           'explicit', 
           'track_pop', 
           'album_pop', 
           'comparative_pop', 
           'danceability', 
           'energy', 
           'loudness', 
           'speechiness', 
           'acousticness', 
           'instrumentalness', 
           'liveness', 
           'valence', 
           'tempo']

In [29]:
# Set up statistics on logistics
track_csv_limit = 10000
# num_output_csvs = 0
# num_tracks = 0
# num_albums = 0
# num_failed_track_calls = 0
# num_failed_album_calls = 0

# Set up the track set to avoid duplicates
# visited_tracks = set()

In [ ]:
rows = []
for csv_id in range(3,333):
    csv = pd.read_csv(f'data/Songs/songs{csv_id}.csv')
    for uri in csv["track_uri"]:
        # Check if the track has already been visited:
        if uri in visited_tracks:
            continue
        else:
            visited_tracks.add(uri)
            
        # Extract the track object
        # If the API fails, iteratively pause for 10 seconds and try again
        sleep_timer = 10
        while True:
            try:
                track_object = sp.track(uri)
                break
            except:
                num_failed_track_calls += 1
                sleep_timer += 10
                time.sleep(sleep_timer)
            if sleep_timer > 120:
                break
        if sleep_timer > 120:
            num_output_csvs += 1
            print("Slept for over 2 minutes and still being limited.")
            print("Saving the data, sleeping for 10 minutes, then continuing.")
            df = pd.DataFrame(rows)
            df.to_csv(f'data/output/clean_spotify_{num_output_csvs}.csv', index=False)
            time.sleep(600)
            break

        # Check if the track is a single
        if track_object['album']['album_type'] == 'single':
            continue
        
        num_tracks += 1
        # Get the track features
        track_id = track_object['id']
        album_id = track_object['album']['id']
        track_count = track_object['album']['total_tracks']
        duration = track_object['duration_ms']
        explicit = track_object['explicit']
        track_pop = track_object['popularity']
        track_number = track_object['track_number']
        
        # Get the album features
        # If the API fails, iteratively pause for 10 seconds and try again
        sleep_timer = 10
        while True:
            try:
                album = sp.album(album_id)
                break
            except:
                num_failed_album_calls += 1
                sleep_timer += 10
                time.sleep(sleep_timer)
            if sleep_timer > 120:
                break

        if sleep_timer > 120:
            num_output_csvs += 1
            print("Slept for over 2 minutes and still being limited.")
            print("Saving the data, sleeping for 10 minutes, then continuing.")
            df = pd.DataFrame(rows)
            df.to_csv(f'data/output/clean_spotify_{num_output_csvs}.csv', index=False)
            time.sleep(600)
            break
            
        num_albums += 1
        album_pop = album['popularity']
        comparative_pop = track_pop - (((album_pop * track_count) - track_pop) / (track_count - 1))
        
        # Get the audio features
        features = sp.audio_features(track_id)[0]
        danceability = features['danceability']
        energy = features['energy']
        loudness = features['loudness']
        speechiness = features['speechiness']
        acousticness = features['acousticness']
        instrumentalness = features['instrumentalness']
        liveness = features['liveness']
        valence = features['valence']
        tempo = features['tempo']
        
        # List the features
        relevant_features = [track_id, album_id, track_number, track_count, duration, explicit, track_pop, 
                             album_pop, comparative_pop, danceability, energy, loudness, speechiness, 
                             acousticness, instrumentalness, liveness, valence, tempo]
        rows.append(dict(zip(columns, relevant_features)))
        
        if num_tracks % track_csv_limit == 0:
            df = pd.DataFrame(rows)
            df.to_csv(f'data/output/clean_spotify_{num_output_csvs}.csv', index=False)
            
            num_output_csvs += 1
            rows = []
            
    print(f"In the data collection from songs/{csv_id}.csv, the following are the statistics:")
    print(f"{num_tracks} were written. {num_albums} were written.")
    print(f"There were {num_failed_track_calls} failed calls to the API for tracks.")
    print(f"There were {num_failed_album_calls} failed calls to the API for albums.")

retrying ...1secs


In [34]:
num_output_csvs += 1

In [33]:
df = pd.DataFrame(rows)
df.to_csv(f'data/output/clean_spotify_{num_output_csvs}.csv', index=False)

In [26]:
len(visited_tracks)

867

In [27]:
len(rows)

782

In [28]:
uri

'spotify:track:4tCtwWceOPWzenK2HAIJSb'